In [1]:
import pandas as pd
import os
from sklearn.model_selection import KFold

#### Mix all 3 hospitals as train with 5 fold Cross val. Then use TCGA as test in the df.

In [4]:

# Load your main CSV file (replace with the actual file path)
main_df = pd.read_csv('/mnt/ncshare/ozkilim/BRCA/data/tasks/HGSOC_multimodal_metastatic_protein_and_phospho.csv')

# Focus on the 'Sample source' and 'slide_id' columns
data = main_df[['Sample Source', 'slide_id']]

# Get unique sample sources
sample_sources = data['Sample Source'].unique()

# Create a KFold object for 5 folds
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Directory to store split files
split_dir = 'split_files'
os.makedirs(split_dir, exist_ok=True)

# Iterate over each sample source to create split files
# Filter data for cross-validation and test sets
cv_data = data['slide_id']
TCGA_data = pd.read_csv('/mnt/ncshare/ozkilim/BRCA/data/tasks/TCGA_platinum_reposnce_test_set.csv')# load tcga data #
test_data = TCGA_data['slide_id']
# Apply 5-fold cross-validation
for fold, (train_index, val_index) in enumerate(kf.split(cv_data)):
    # Extract train, val, and test slide IDs
    train_ids = cv_data.iloc[train_index].tolist()
    val_ids = cv_data.iloc[val_index].tolist()
    test_ids = test_data.tolist()

    # Determine the maximum length among train, val, and test lists
    max_len = max(len(train_ids), len(val_ids), len(test_ids))

    # Extend lists to have the same length
    train_ids.extend([""] * (max_len - len(train_ids)))
    val_ids.extend([""] * (max_len - len(val_ids)))
    test_ids.extend([""] * (max_len - len(test_ids)))

    # Create the split DataFrame
    split_df = pd.DataFrame({'train': train_ids[:max_len], 'val': val_ids[:max_len], 'test': test_ids[:max_len]})

    # Save the split file
    split_filename = f'/mnt/ncshare/ozkilim/BRCA/splits/TCGA_metastatic/splits_{fold}.csv'
    # split_filepath = os.path.join(split_dir, split_filename)
    split_df.to_csv(split_filename, index=True)

print("Split files created successfully.")

348
348
348
348
348
Split files created successfully.


In [4]:
# Create merged csv file for getting labels and omics data 

HGSCO_main = pd.read_csv("/mnt/ncshare/ozkilim/BRCA/data/tasks/HGSOC_multimodal_protein_and_phospho.csv")

# merge together with common columns... 

common_columns = HGSCO_main.columns.intersection(TCGA_data.columns)
# Select only common columns from both dataframes
HGSCO_main = HGSCO_main[common_columns]
TCGA_data = TCGA_data[common_columns]

# Concatenate the rows
combined_df = pd.concat([HGSCO_main, TCGA_data], ignore_index=True)


/home/ozkilim/anaconda3/envs/clam/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3331: DtypeWarning: Columns (8807,8811) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [8]:
combined_df.to_csv("/mnt/ncshare/ozkilim/BRCA/data/tasks/HGSOC_TCGA_merged_all.csv",index=None)